#Importing in drive direct from kaggle

In [ ]:
'''from google.colab import drive
drive.mount('/content/gdrive')'''

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
'''import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive
#changing the working directory
%cd /content/gdrive/My Drive/Kaggle
#Check the present working directory using pwd command
!kaggle datasets download -d bing101/friendshipgoals'''

#Dataset Link for downloading
https://www.kaggle.com/bing101/friendshipgoals/download

#Importing Essential Libraries

In [ ]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array,array_to_img
 
%matplotlib inline
from scipy.stats import norm
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense,Layer,Lambda,Add,Multiply,Input,Conv2D,MaxPool2D,Flatten,Conv2DTranspose,Reshape,Dropout,GlobalAveragePooling2D,GaussianNoise
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.utils import plot_model
from PIL import Image
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import mse
import pandas as pd
import shutil
from tensorflow.keras.applications.resnet_v2 import ResNet50V2,preprocess_input

#Unzipping Dataset

In [ ]:
'''dataset_path = "/content/drive/My Drive/Kaggle/friendshipgoals.zip"
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("/content/drive/My Drive/Kaggle")
zip_object.close()'''

#Assigning paths towards directories

In [ ]:
train_path = "/content/drive/My Drive/Kaggle/data/train"
test_path = "/content/drive/My Drive/Kaggle/data/test_out"
validate_path = "/content/drive/My Drive/Kaggle/data/validate"

# Run Once to segregate 20% of train set to validate set

In [ ]:
'''
#(((To shift from train folder to validate folder)))
adu_shift = os.listdir(train_path+"/Adults")
size = int(0.2*len(adu_shift))
for f in adu_shift[:size]:
    shutil.move(train_path+"/Adults/"+f,validate_path+"/Adults")

teen_shift = os.listdir(train_path+"/Teenagers")
size = int(0.2*len(teen_shift))
for f in teen_shift[:size]:
    shutil.move(train_path+"/Teenagers/"+f,validate_path+"/Teenagers")

tod_shift = os.listdir(train_path+"/Toddler")
size = int(0.2*len(tod_shift))
for f in tod_shift[:size]:
    shutil.move(train_path+"/Toddler/"+f,validate_path+"/Toddler")'''

#Run to push_back validate set images back to train set

In [ ]:
'''
# To shift from validate folder back to test folder
adu_shift = os.listdir("/content/drive/My Drive/Kaggle/data/validate/Adults/")
for f in adu_shift:
    shutil.move(validate_path+"/Adults/"+f,train_path+"/Adults")
teen_shift = os.listdir("/content/drive/My Drive/Kaggle/data/validate/Teenagers/")
for f in teen_shift:
    shutil.move(validate_path+"/Teenagers/"+f,train_path+"/Teenagers")
tod_shift = os.listdir("/content/drive/My Drive/Kaggle/data/validate/Toddler/")
for f in tod_shift:
    shutil.move(validate_path+"/Toddler/"+f,train_path+"/Toddler")
'''

#Label Encoding the Images
```
e.g.
Adults - 0,
Teenagers - 1,
Toddler - 2.
```

In [ ]:
lst = ['Adults','Teenagers','Toddler']
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
LE.fit(lst)

LabelEncoder()

#Initializing ImageDataGenerator for all train,test,validate

In [ ]:
data_gen_train = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    preprocessing_function = preprocess_input)
data_gen_valid = ImageDataGenerator(
    preprocessing_function = preprocess_input)
data_gen_test = ImageDataGenerator(
    preprocessing_function = preprocess_input)

In [ ]:
train_generator = data_gen_train.flow_from_directory(train_path, target_size=(256,256), class_mode='categorical', batch_size=32)
validation_generator = data_gen_valid.flow_from_directory(validate_path, target_size=(256,256), class_mode='categorical',batch_size=32)
test_generator = data_gen_test.flow_from_directory(test_path, target_size=(256,256),class_mode=None,shuffle=False)

Found 2243 images belonging to 3 classes.
Found 560 images belonging to 3 classes.
Found 314 images belonging to 1 classes.


#Initializing base_model

In [ ]:
img_shape = (256,256,3)
fine_tune_at = 185
base_model = ResNet50V2(include_top=False,weights='imagenet',input_shape=img_shape )
base_model.trainable = True
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

#Adding extra layers and creating final Model

In [ ]:
global_average_layer = GlobalAveragePooling2D()(base_model.output)
lay_1 = Dense(units=64, activation='relu')(global_average_layer)
d1 = Dropout(0.2)(lay_1)
lay_2 = Dense(units=32, activation='relu')(d1)
d2 = Dropout(0.2)(lay_2)
prediction_layer = Dense(units=3, activation='softmax')(d2)
model = Model(inputs=base_model.input, outputs=prediction_layer)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

#Model fitting

In [ ]:
history = model.fit(train_generator, epochs=10,steps_per_epoch=30,validation_data = validation_generator)

#Model Prediction and to_csv file write

In [ ]:
ans = model.predict(test_generator)
solution_index = np.argmax(ans,axis=1)
names = os.listdir(test_path+'/test')

match = {}
for i,x in enumerate(names):
    match[x] = LE.inverse_transform([solution_index[i] ])[0]

labels = pd.read_csv('/content/drive/My Drive/Kaggle/data/Test.csv')
img_names = labels.iloc[:,:].values
img_names = img_names.reshape(-1,)

frame = []
for x in img_names:
    frame.append([x,match[x]])

df = pd.DataFrame(frame,columns=["Filename","Category"])
letgo = df.to_csv('./pred1.csv',index=False)